# 机器翻译

大家应该都使用过谷歌翻译或者百度翻译或者有道词典等等翻译工具，它们内部的技术就是机器翻译，通过训练深度神经网络来让机器学会一门语言翻译能力。

本次实战编程实现的神经网络也可以用实现语言翻译，但是需要非常非常大的训练数据集和非常非常长的训练时间，所以处于教学目的，本次我们只用它来实现日期格式的翻译。日期的格式有很多种，仅仅是英文日期都有很多种表现格式，例如"the 29th of August 1958", "03/30/1968", "24 JUNE 1987"等等。一般来说电脑里的时间格式是"2009-06-25"，我们在这里将其称为电脑日期格式，将其它类型的格式称为人类日期格式。本次实战编程的目的就是要通过神经网络来将人类日期格式翻译成电脑日期格式。其实这个与英语翻译成汉语是同一个道理，只不过简单一些而已，仅仅需要少量的训练集就可以搞定了。


首先按照下面的指令安装新的工具库。

1，打开Anaconda prompt

2，执行activate tensorflow命令

3，执行pip install faker==2.0.3命令

4，执行pip install tqdm命令

5，执行pip install babel命令

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 1 - 翻译时间格式

### 1.1 - 数据集

下面的代码将加载10000个样本的数据集

In [2]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 5800.74it/s]


In [3]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('08 jul 2008', '2008-07-08'),
 ('8 sep 1999', '1999-09-08'),
 ('thursday january 1 1981', '1981-01-01')]

- `dataset`: 这是一个tuples类型的list列表，每一项就是一个时间格式对（人类日期格式，电脑时间格式）,上面的代码打印出了前面10个时间格式对。
- `human_vocab`: 这是一个字典，用于将人类日期格式里面的每一个字符转换成一个对应的数字索引。
- `machine_vocab`: 这是一个字典，用于将电脑日期格式里面的每一个字符转换成一个对应的数字索引。注意，这个索引与上面的人类日期的索引不需要一一对应。
- `inv_machine_vocab`: 这是一个字典，作用是与machine_vocab相反的，就是将索引转换成字符。 

In [4]:
# 下面的代码会将数据集里面的字符都转换成索引格式，并且拆分出X和Y，同时转换成one-hot形式。
Tx = 30 # 这里是假设在人类日期格式中最多有30个字符，如果超过30，那么会被截断。
Ty = 10 # 电脑格式"YYYY-MM-DD"中字符数量是固定的，就是10个。
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


- `X`: 输入X。这里面的人类格式日期的字符都是以索引来表示的了，而且每个日期都加了padding或被截断了，以保证长度都是30，所以X的维度是`X.shape = (m, Tx)`
- `Y`: 真实标签Y。这里面的电脑格式日期的字符也都是以索引来表示的了。维度是`Y.shape = (m, Ty)`. 
- `Xoh`: X的one-hot版本。也就是用一个长度为len(human_vocab)的one-hot向量来表示某个字符。所以维度是 `Xoh.shape = (m, Tx, len(human_vocab))`
- `Yoh`: Y的one-hot版本。维度是 `Yoh.shape = (m, Tx, len(machine_vocab))`. 这里的`len(machine_vocab) = 11`因为电脑格式日期中只有'-'和0-9这11个字符。


下面的代码将某个日期的不同表现方式打印出来了，有助于大家理解X和Xoh等

In [5]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2 - 使用注意力模型来实现机器翻译

当我翻译一篇英文文档时，我通常是先将文档看一遍，然后再从头开始一段一段的进行翻译。也就是说，当我进行翻译时，我会把大部分的注意力集中在当前要翻译的部分上。我们学过的注意力模型也是这个机制，它会告诉神经网络模型在翻译的某一个时刻应该把注意力放在句子的哪些部位上。


### 2.1 - 注意力机制

下面的左图是一个注意力模型，右图展示了在某一个解码网络的时间步时，与其相关的注意力权重$\alpha^{\langle t, t' \rangle}$是如何被计算出来的。最后这些注意力权重和相应的编码网络的激活值a会组成一个变量$context^{\langle t \rangle}$，然后输入到解码网络的这个时间步中。这里要再次提醒大家要区分注意力权重$\alpha$和激活值a！另外要注意的是我们使用t表示解码网络中的时间步编号，使用t'表示编码网络中的时间步编号。

<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>
<caption><center> **图 1**</center></caption>


下面是一些对上图的解释: 

- 左图中有两个LSTM，下面的双向LSTM是编码网络，由于它在注意力机制之前，所以也可以称为*pre-attention* Bi-LSTM。上面的LSTM是解码网络，因为在注意力机制后面，所以也被称为*post-attention* LSTM。编码网络有$T_x$个时间步，本例中是30; 解码网络中有$T_y$个时间步。 

- 我们知道LSTM的每个时间步都会产生一个激活值$s^{\langle t \rangle}$和一个$c^{\langle t \rangle}$。如果不懂我在说什么，那么请复习一下前面关于LSTM的文章（5.1.10）。在前面的文章中因为我们只使用了一个简单的RNN来举例，所以那时使用$c^{\langle t \rangle}$来表示上图中的$context^{\langle t \rangle}$，但是在本文档中，我们使用了LSTM，所以就会出现两个c，为了避免冲突，本文档中使用了$context^{\langle t \rangle}$。另外，由于日期中各字符的关联不是很大，所以在解码网络中没有将上一个时间步的输出值$y^{\langle t-1 \rangle}$传递到下一个时间步去。

- 编码网络是个双向的网络，所以$a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$,也就是包含了一个正向的激活值和反向的激活值。

- 右图中使用了两个Keras的函数。使用`RepeatVector`来将$s^{\langle t-1 \rangle}$'复制了 $T_x$次，然后使用`Concatenation`来将$s^{\langle t-1 \rangle}$和$a^{\langle t \rangle}$结合成变量$e^{\langle t, t'}$,最后输入到一个小型神经网络中来产生注意力权重$\alpha^{\langle t, t' \rangle}$。


In [6]:
# 为了实现右图，我们定义一些全局操作对象，在后面的函数中就可以多次调用这些对象了。
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
# 这个densor对象代表了一个dense网络层，
# 由于这个对象是全局的，所以在每次调用它时，都是在训练同一套相关参数，这正是我们需要的。
# 因为不停地训练同一套参数，那么网络就会越来越聪明，输出的注意力权重就会越来越合理。
densor = Dense(1, activation = "relu") 
activator = Activation(softmax, name='attention_weights') 
dotor = Dot(axes = 1)

In [7]:
# 实现上面右图的功能，也就是为解码网络的某一个时间步计算相应的context输入

def one_step_attention(a, s_prev):
    """    
    参数:
    a -- 编码网络的所有时间步的激活值，维度是(m, Tx, 2*n_a)，这里n_a是神经元个数，乘以2是因为是双向网络。
    s_prev -- 解码网络中前一个时间步激活值，维度是 (m, n_s)
    
    返回值:
    context -- 这个结果会输入到解码网络的时间步中去。
    """
    
    # 使用repeator对象将s_prev复制出多份来，使其维度成为 (m, Tx, n_s)，这样一来才能与激活值a进行组合 
    s_prev = repeator(s_prev)
    # 使用concatenator对象将a和s_prev连接起来
    concat = concatenator([a, s_prev])
    # 将其传入到一个全连接网络层中去，得到注意力权重alphas
    e = densor(concat)
    alphas = activator(e)
    # 将激活值和注意力权重相乘然后相加，得到context
    context = dotor([alphas, a])
    
    return context

In [8]:
# 定义一些构成模型时需要的全局对象，同理，在后面对这些对象进行调用时，始终在训练着同一套参数，这正是我们需要的
# 因为每个时间步的参数应该是共享一套的。
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [9]:
# 实现前面的左图。

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    参数:
    Tx -- 输入句子的最大长度，本文档我们使用30
    Ty -- 输出句子的最大长度，是11
    n_a -- 编码网络中每个LSTM的神经元个数
    n_s -- 解码网络中每个LSTM的神经元个数
    human_vocab_size -- 人类格式日期的词表的大小
    machine_vocab_size -- 电脑格式日期的词表的大小

    返回值:
    model -- Keras模型实例
    """
    
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    # 下面这一句简短的代码就将编码网络给构建好了。此时你会再次感叹，Keras太方便了。
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # 用for循环来执行解码网络中的每一个时间步
    for t in range(Ty):
    
        # 调用前面我们实现的one_step_attention函数来为当前时间步计算出相应的context输入。
        context = one_step_attention(a, s)
        
        # 执行这个时间步。得出新的s和c
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])
        
        # 得出一个时间步的预测值
        out = output_layer(s)
        
        outputs.append(out)
    
    # 创建keras实例
    model = Model(inputs = [X, s0, c0], outputs = outputs)
    
    return model

In [10]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

下面的代码会输出这个模型的一些信息

In [11]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 30, 37)        0                                            
____________________________________________________________________________________________________
s0 (InputLayer)                  (None, 128)           0                                            
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 30, 128)       52224       input_1[0][0]                    
____________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)   (None, 30, 128)       0           s0[0][0]                         
                                                                   lstm_1[0][0]            

In [12]:
#编译这个模型
out = model.compile(optimizer=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01),
                    metrics=['accuracy'],
                    loss='categorical_crossentropy')

In [13]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

开始训练模型

In [16]:
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10
10000/10000 [==============================] - 49s - loss: 3.5670 - dense_2_loss_1: 0.0603 - dense_2_loss_2: 0.0542 - dense_2_loss_3: 0.3003 - dense_2_loss_4: 0.6077 - dense_2_loss_5: 0.0057 - dense_2_loss_6: 0.0908 - dense_2_loss_7: 0.8481 - dense_2_loss_8: 0.0056 - dense_2_loss_9: 0.5972 - dense_2_loss_10: 0.9971 - dense_2_acc_1: 0.9767 - dense_2_acc_2: 0.9777 - dense_2_acc_3: 0.8784 - dense_2_acc_4: 0.7991 - dense_2_acc_5: 1.0000 - dense_2_acc_6: 0.9721 - dense_2_acc_7: 0.7160 - dense_2_acc_8: 1.0000 - dense_2_acc_9: 0.7629 - dense_2_acc_10: 0.6296    
Epoch 2/10
10000/10000 [==============================] - 49s - loss: 2.6884 - dense_2_loss_1: 0.0483 - dense_2_loss_2: 0.0415 - dense_2_loss_3: 0.2513 - dense_2_loss_4: 0.3833 - dense_2_loss_5: 0.0045 - dense_2_loss_6: 0.0785 - dense_2_loss_7: 0.6556 - dense_2_loss_8: 0.0046 - dense_2_loss_9: 0.4952 - dense_2_loss_10: 0.7255 - dense_2_acc_1: 0.9810 - dense_2_acc_2: 0.9826 - dense_2_acc_3: 0.8889 - dense_2_acc_4: 0.8849 - d

下面用这个模型来翻译一些日期

In [17]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1979-05-03
source: 5 April 09
output: 2009-04-06
source: 21th of August 2016
output: 2016-08-14
source: Tue 10 Jul 2007
output: 2007-07-10
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2011-03-03
source: March 3rd 2001
output: 2011-03-03
source: 1 March 2001
output: 2011-03-01
